In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm
from ipywidgets import interact, FloatSlider, Dropdown, fixed
from IPython.display import display


# Словарь файлов и соответствующих им названий столбцов
files = {
    'tksg.csv': 'TCSG',
    'yandex.csv': 'YDEX',
    'vk.csv': 'VKCO',
    'rostelecom.csv': 'RTKM',
    'usd_rub.csv': 'USD_RUB',
    'brent.csv': 'Brent',
    'bitcoin.csv': 'BTC_USD'
}

# Чтение и отображение первых и последних 5 строк каждого файла и статистики
for file_name, column_name in files.items():
    try:
        df = pd.read_csv(file_name)
        print(f"\nСтатистика файла {file_name}:")
        print(f"Количество строк: {len(df)}")
        print(f"Количество столбцов: {len(df.columns)}")
        
        print("\nПервые 5 строк файла:")
        print(df.head())
        print("\nПоследние 5 строк файла:")
        print(df.tail())
        print("-" * 80)
    except FileNotFoundError:
        print(f"Файл {file_name} не найден")
    except Exception as e:
        print(f"Ошибка при чтении файла {file_name}: {str(e)}")

In [ ]:
# 1 обьединение данных в единый временной ряд методом inner join

# Функция для конвертации строк с числами в float
def convert_to_float(value):
    try:
        # Удаляем точку как разделитель тысяч и заменяем запятую на точку для десятичного разделителя
        cleaned_value = str(value).replace('.', '').replace(',', '.')
        return float(cleaned_value)
    except (ValueError, TypeError):
        return value  # Возвращаем исходное значение, если конвертация не удалась

# Создание единого датафрейма final_df
dfs = []
for file_name, column_name in files.items():
    try:
        df = pd.read_csv(file_name)
        # Выбираем только нужные столбцы и переименовываем столбец с ценой
        df = df[['Дата', 'Цена']].rename(columns={'Цена': column_name})
        dfs.append(df)
    except FileNotFoundError:
        print(f"Файл {file_name} не найден")
    except Exception as e:
        print(f"Ошибка при чтении файла {file_name}: {str(e)}")

# Объединение всех датафреймов
if dfs:
    # Начинаем с первого датафрейма
    final_df = dfs[0]
    # Последовательно объединяем с остальными
    for df in dfs[1:]:
        final_df = pd.merge(final_df, df, on='Дата', how='inner')
    
    # Сортировка по дате
    final_df['Дата'] = pd.to_datetime(final_df['Дата'], format='%d.%m.%Y')
    final_df = final_df.sort_values('Дата')
    
    # Преобразование данных в числовой формат
    for column in final_df.columns:
        if column != 'Дата':
            final_df[column] = final_df[column].apply(convert_to_float)
    
    # Вывод информации и статистики до очистки
    print("\nИтоговый датафрейм:")
    print("\nПервые 5 строк:")
    print(final_df.head())
    print("\nПоследние 5 строк:")
    print(final_df.tail())
    print("\nИнформация о датафрейме:")
    print(final_df.info())
    print("\nСтатистика датафрейма:")
    print(final_df.describe())
    
    # Построение графика
    plt.figure(figsize=(15, 8))
    
    # Нормализация данных (приведение к базе 100)
    normalized_df = final_df.copy()
    for column in normalized_df.columns:
        if column != 'Дата':
            normalized_df[column] = normalized_df[column] / normalized_df[column].iloc[0] * 100
    
    # Построение графика для каждого актива
    for column in normalized_df.columns:
        if column != 'Дата':
            plt.plot(normalized_df['Дата'], normalized_df[column], label=column)
    
    plt.title('Динамика активов (база 100)')
    plt.xlabel('Дата')
    plt.ylabel('Значение (база 100)')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
else:
    print("Не удалось создать датафрейм из-за ошибок при чтении файлов")

In [ ]:
# Очистка от выбросов (менее строгая: удаляем строки, где ВСЕ столбцы являются выбросами)
def remove_outliers(df, columns):
    mask = pd.Series(True, index=df.index)  # Изначально все строки включены
    for col in columns:
        Q1 = df[col].quantile(0.25)
        Q3 = df[col].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        # Обновляем маску: строка остается, если значение в столбце НЕ является выбросом
        mask = mask & (df[col].between(lower_bound, upper_bound))
    return df[mask]

# Очистка от выбросов
columns_to_clean = list(files.values())
cleaned_df = remove_outliers(final_df, columns_to_clean)

# Статистика после очистки
print("\nИтоговый датафрейм после очистки от выбросов:")
print("\nПервые 5 строк:")
print(cleaned_df.head())
print("\nПоследние 5 строк:")
print(cleaned_df.tail())
print("\nСтатистика после очистки:")
print(cleaned_df.info())
# print(cleaned_df.describe())

# Построение графика для очищенных данных
plt.figure(figsize=(15, 8))

# Нормализация данных (приведение к базе 100) для очищенного датафрейма
normalized_df = cleaned_df.copy()
for column in normalized_df.columns:
    if column != 'Дата':
        normalized_df[column] = normalized_df[column] / normalized_df[column].iloc[0] * 100

# Построение графика для каждого актива
for column in normalized_df.columns:
    if column != 'Дата':
        plt.plot(normalized_df['Дата'], normalized_df[column], label=column)

plt.title('Динамика активов (база 100, после очистки от выбросов)')
plt.xlabel('Дата')
plt.ylabel('Значение (база 100)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

In [ ]:
# 3. Нормализация (Z-нормализация)
# normalized_df = final_df.copy()
# for col in columns_to_clean:
#     mean = final_df[col].mean()
#     std = final_df[col].std()
#     normalized_df[col] = (final_df[col] - mean) / std
#     print(f"\nСтатистика для {col} (до нормализации): mean = {mean:.2f}, std = {std:.2f}")


normalized_df = cleaned_df.copy()
for col in columns_to_clean:
    mean = cleaned_df[col].mean()
    std = cleaned_df[col].std()
    normalized_df[col] = (cleaned_df[col] - mean) / std
    print(f"\nСтатистика для {col} (до нормализации): mean = {mean:.2f}, std = {std:.2f}")

print("\nПример нормализованного датафрейма (первые 5 строк):")
print(normalized_df.head())
print("\nСтатистика после нормализации:")
print(normalized_df.describe())

In [ ]:
# 4. тест шапиро уилка
factors = ['TCSG', 'YDEX', 'VKCO', 'RTKM', 'USD_RUB', 'Brent', 'BTC_USD']
normality_results = []
distribution_stats = []

for factor in factors:
    data = normalized_df[factor].dropna()
    # Тест Шапиро-Уилка
    stat, p_value = stats.shapiro(data)
    conclusion = "Не соответствует нормальному распределению" if p_value < 0.05 else "Соответствует нормальному распределению"
    normality_results.append([factor, stat, p_value, conclusion])

    # Характеристики распределения
    skewness = stats.skew(data)
    kurtosis = stats.kurtosis(data)
    mean = data.mean()
    median = data.median()
    std = data.std()
    min_val = data.min()
    max_val = data.max()
    q1 = data.quantile(0.25)
    q3 = data.quantile(0.75)
    
    # Интерпретация асимметрии
    skewness_comment = (
        "Сильная правосторонняя асимметрия" if skewness > 1 else
        "Сильная левосторонняя асимметрия" if skewness < -1 else
        "Умеренная асимметрия" if abs(skewness) > 0.5 else
        "Приблизительно симметричное распределение"
    )
    
    # Интерпретация куртозиса
    kurtosis_comment = (
        "Тяжёлые хвосты (много выбросов)" if kurtosis > 3 else
        "Лёгкие хвосты (меньшая вероятность экстремальных значений, что положительно для моделирования)" if kurtosis < -1 else
        "Нормальные хвосты (умеренное количество выбросов)"
    )
    
    # Дополнительные комментарии для факторов
    factor_comment = (
        "Высокая волатильность из-за влияния криптовалютного рынка и санкций." if factor == 'TCSG' else
        "Зависимость от сырьевых рынков (нефть) и макроэкономической среды." if factor in ['YDEX', 'VKCO', 'RTKM'] else
        "Волатильность связана с экономическими и геополитическими факторами." if factor == 'USD_RUB' else
        "Зависимость от мировых сырьевых рынков." if factor == 'Brent' else
        "Высокая волатильность из-за спекулятивного характера криптовалют."
    )
    
    distribution_stats.append([
        factor, mean, median, std, skewness, kurtosis, min_val, max_val, q1, q3,
        skewness_comment, kurtosis_comment, factor_comment
    ])

# Формирование таблиц результатов
normality_df = pd.DataFrame(normality_results, columns=['Фактор', 'Статистика Шапиро-Уилка', 'p-value', 'Нормальность'])
dist_stats_df = pd.DataFrame(distribution_stats, columns=[
    'Фактор', 'Среднее', 'Медиана', 'Стд. откл.', 'Асимметрия', 'Куртозис',
    'Мин.', 'Макс.', 'Q1', 'Q3', 'Асимметрия (пояснение)', 'Куртозис (пояснение)', 'Комментарий'
]).round(4)

# Вывод таблиц
print("\nРезультаты теста Шапиро-Уилка:")
print(normality_df.to_string(index=False))
# print("\nХарактеристики распределения:")
# print(dist_stats_df.to_string(index=False))

In [ ]:
# 5. Корреляционный анализ 
def analyze_correlations():
    print("\n=== Корреляционный анализ (метод Спирмена) ===")
    
    # Корреляция Спирмена
    correlation_matrix = normalized_df[columns_to_clean].corr(method='spearman')
    
    # # Форматирование матрицы для вывода
    # corr_df = correlation_matrix.round(3)  # Округляем до 3 знаков после запятой
    # print("\nМатрица корреляции (Спирмен):")
    # display(corr_df.style.background_gradient(cmap='coolwarm', axis=None).set_caption('Корреляционная матрица (Спирмен)'))

    # Визуализация корреляции
    plt.figure(figsize=(10, 8), dpi=100)
    plt.matshow(correlation_matrix, cmap='coolwarm', fignum=1)
    plt.xticks(range(len(columns_to_clean)), columns_to_clean, rotation=45, ha='left')
    plt.yticks(range(len(columns_to_clean)), columns_to_clean)
    for (i, j), val in np.ndenumerate(correlation_matrix):
        plt.text(j, i, f'{val:.2f}', ha='center', va='center', color='black', fontsize=10)
    plt.title('Корреляция Спирмена', fontsize=14, pad=20)
    plt.colorbar(label='Коэффициент корреляции')
    plt.tight_layout()
    plt.show()

# Выполняем корреляционный анализ
analyze_correlations()

In [ ]:
# 6. Модель линейной регрессии
dependent_vars = ['TCSG', 'YDEX', 'VKCO', 'RTKM']
independent_vars = ['USD_RUB', 'Brent', 'BTC_USD']

results_normalized = {}  # Линейная регрессия
X_normalized = normalized_df[independent_vars].dropna()

for y_var in dependent_vars:
    y = normalized_df[y_var].dropna()
    common_index = X_normalized.index.intersection(y.index)
    X_subset = X_normalized.loc[common_index]
    y_subset = y.loc[common_index]
    
    # Разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)
    
    # Линейная регрессия
    model_lr = LinearRegression()
    model_lr.fit(X_train, y_train)
    y_pred_lr = model_lr.predict(X_test)
    mae_lr = np.mean(np.abs(y_test - y_pred_lr))
    mse_lr = np.mean((y_test - y_pred_lr) ** 2)
    
    results_normalized[y_var] = {
        'y_actual': y_test,
        'y_pred': y_pred_lr,
        'X_test': X_test,
        'model': model_lr,
        'coefficients': dict(zip(independent_vars, model_lr.coef_)),
        'intercept': model_lr.intercept_,
        'MAE': mae_lr,
        'MSE': mse_lr
    }
    
# Формирование таблицы регрессии
results_data = []
for y_var in dependent_vars:
    coeffs = results_normalized[y_var]['coefficients']
    results_data.append([
        y_var,
        coeffs.get('USD_RUB', 0),
        coeffs.get('Brent', 0),
        coeffs.get('BTC_USD', 0),
        results_normalized[y_var]['intercept'],
        results_normalized[y_var]['MAE'],
        results_normalized[y_var]['MSE']
    ])
results_df = pd.DataFrame(
    results_data,
    columns=['Компания', 'USD_RUB (β1)', 'Brent (β2)', 'BTC_USD (β3)', 'Intercept', 'MAE', 'MSE']
).round(4)

print("\nРезультаты линейной регрессии (на тестовой выборке):")
print(results_df.to_string(index=False))

# Анализ ошибок регрессии
print("\nАнализ ошибок линейной регрессии (на тестовой выборке):")
for y_var in dependent_vars:
    errors = results_normalized[y_var]['y_actual'] - results_normalized[y_var]['y_pred']
    error_mean = errors.mean()
    error_std = errors.std()
    error_skewness = stats.skew(errors)
    error_kurtosis = stats.kurtosis(errors)
    
    stat, p = stats.shapiro(errors)
    normality_comment = "Ошибки не соответствуют нормальному распределению" if p < 0.05 else "Ошибки соответствуют нормальному распределению"
    
    error_mean_comment = (
        f"Ошибки в среднем {'смещены вверх' if error_mean > 0.1 else 'смещены вниз' if error_mean < -0.1 else 'близки к нулю'} "
        f"(среднее: {error_mean:.4f})."
    )
    error_std_comment = (
        f"{'Высокая вариабельность ошибок' if error_std > 1 else 'Умеренная вариабельность ошибок' if error_std > 0.5 else 'Низкая вариабельность ошибок'} "
        f"(стандартное отклонение: {error_std:.4f})."
    )
    error_skewness_comment = (
        "Сильная правосторонняя асимметрия ошибок" if error_skewness > 1 else
        "Сильная левосторонняя асимметрия ошибок" if error_skewness < -1 else
        "Умеренная асимметрия ошибок" if abs(error_skewness) > 0.5 else
        "Ошибки симметричны"
    )
    error_kurtosis_comment = (
        "Тяжёлые хвосты ошибок (много выбросов)" if error_kurtosis > 3 else
        "Лёгкие хвосты ошибок (меньшая вероятность экстремальных значений)" if error_kurtosis < -1 else
        "Нормальные хвосты ошибок"
    )
    
    print(f"\nХарактеристики ошибок для {y_var}:")
    print(f"- Среднее: {error_mean_comment}")
    print(f"- Стандартное отклонение: {error_std_comment}")
    print(f"- Асимметрия: {error_skewness_comment} (значение: {error_skewness:.4f})")
    print(f"- Куртозис: {error_kurtosis_comment} (значение: {error_kurtosis:.4f})")
    print(f"- Шапиро-Уилка: stat={stat:.4f}, p={p:.4f}, {normality_comment}")
    
    # Гистограмма ошибок
    plt.figure(figsize=(8, 6))
    sns.histplot(errors, kde=True, bins=50, color='purple')
    plt.title(f'Распределение ошибок для {y_var} (линейная регрессия)')
    plt.xlabel('Ошибка (реальное - предсказанное)')
    plt.ylabel('Частота')
    plt.xlim(-5, 5)
    plt.grid(True)
    plt.show()
    
    # График предсказанных vs реальных значений
    y_actual = results_normalized[y_var]['y_actual']
    y_pred = results_normalized[y_var]['y_pred']
    plt.figure(figsize=(8, 6))
    plt.scatter(y_actual, y_pred, color='blue', alpha=0.5)
    plt.plot([y_actual.min(), y_actual.max()], [y_actual.min(), y_actual.max()], 'r--', lw=2)
    plt.title(f'Предсказанные vs Реальные значения для {y_var} (линейная регрессия)')
    plt.xlabel('Реальные значения (Z-нормализованные)')
    plt.ylabel('Предсказанные значения (Z-нормализованные)')
    plt.grid(True)
    plt.tight_layout()
    plt.show()

In [ ]:
# 7. Модель случайного леса
dependent_vars = ['TCSG', 'YDEX', 'VKCO', 'RTKM']
independent_vars = ['USD_RUB', 'Brent', 'BTC_USD']

results_normalized_rf = {}  # Случайный лес
X_normalized = normalized_df[independent_vars].dropna()

for y_var in dependent_vars:
    y = normalized_df[y_var].dropna()
    common_index = X_normalized.index.intersection(y.index)
    X_subset = X_normalized.loc[common_index]
    y_subset = y.loc[common_index]
    
    # Разбиение данных
    X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)
    
    # Случайный лес
    model_rf = RandomForestRegressor(n_estimators=100, random_state=42)
    model_rf.fit(X_train, y_train)
    y_pred_rf = model_rf.predict(X_test)
    mae_rf = np.mean(np.abs(y_test - y_pred_rf))
    mse_rf = np.mean((y_test - y_pred_rf) ** 2)
    
    results_normalized_rf[y_var] = {
        'y_actual': y_test,
        'y_pred': y_pred_rf,
        'X_test': X_test,
        'model': model_rf,
        'feature_importances': dict(zip(independent_vars, model_rf.feature_importances_)),
        'MAE': mae_rf,
        'MSE': mse_rf
    }
    
# Формирование таблицы для случайного леса
results_data_rf = []
for y_var in dependent_vars:
    importances = results_normalized_rf[y_var]['feature_importances']
    results_data_rf.append([
        y_var,
        importances.get('USD_RUB', 0),
        importances.get('Brent', 0),
        importances.get('BTC_USD', 0),
        results_normalized_rf[y_var]['MAE'],
        results_normalized_rf[y_var]['MSE']
    ])
results_df_rf = pd.DataFrame(
    results_data_rf,
    columns=['Компания', 'USD_RUB (важность)', 'Brent (важность)', 'BTC_USD (важность)', 'MAE', 'MSE']
).round(4)

print("\nРезультаты случайного леса (на тестовой выборке):")
print(results_df_rf.to_string(index=False))

# Анализ ошибок случайного леса
print("\nАнализ ошибок случайного леса (на тестовой выборке):")
for y_var in dependent_vars:
    errors = results_normalized_rf[y_var]['y_actual'] - results_normalized_rf[y_var]['y_pred']
    error_mean = errors.mean()
    error_std = errors.std()
    error_skewness = stats.skew(errors)
    error_kurtosis = stats.kurtosis(errors)
    
    stat, p = stats.shapiro(errors)
    normality_comment = "Ошибки не соответствуют нормальному распределению" if p < 0.05 else "Ошибки соответствуют нормальному распределению"
    
    error_mean_comment = (
        f"Ошибки в среднем {'смещены вверх' if error_mean > 0.1 else 'смещены вниз' if error_mean < -0.1 else 'близки к нулю'} "
        f"(среднее: {error_mean:.4f})."
    )
    error_std_comment = (
        f"{'Высокая вариабельность ошибок' if error_std > 1 else 'Умеренная вариабельность ошибок' if error_std > 0.5 else 'Низкая вариабельность ошибок'} "
        f"(стандартное отклонение: {error_std:.4f})."
    )
    error_skewness_comment = (
        "Сильная правосторонняя асимметрия ошибок" if error_skewness > 1 else
        "Сильная левосторонняя асимметрия ошибок" if error_skewness < -1 else
        "Умеренная асимметрия ошибок" if abs(error_skewness) > 0.5 else
        "Ошибки симметричны"
    )
    error_kurtosis_comment = (
        "Тяжёлые хвосты ошибок (много выбросов)" if error_kurtosis > 3 else
        "Лёгкие хвосты ошибок (меньшая вероятность экстремальных значений)" if error_kurtosis < -1 else
        "Нормальные хвосты ошибок"
    )
    
    # print(f"\nХарактеристики ошибок для {y_var}:")
    # print(f"- Среднее: {error_mean_comment}")
    # print(f"- Стандартное отклонение: {error_std_comment}")
    # print(f"- Асимметрия: {error_skewness_comment} (значение: {error_skewness:.4f})")
    # print(f"- Куртозис: {error_kurtosis_comment} (значение: {error_kurtosis:.4f})")
    # print(f"- Шапиро-Уилка: stat={stat:.4f}, p={p:.4f}, {normality_comment}")
    
    # # Гистограмма ошибок
    # plt.figure(figsize=(8, 6))
    # sns.histplot(errors, kde=True, bins=50, color='purple')
    # plt.title(f'Распределение ошибок для {y_var} (случайный лес)')
    # plt.xlabel('Ошибка (реальное - предсказанное)')
    # plt.ylabel('Частота')
    # plt.xlim(-5, 5)
    # plt.grid(True)
    # plt.show()
    
    # # График предсказанных vs реальных значений
    # y_actual = results_normalized_rf[y_var]['y_actual']
    # y_pred = results_normalized_rf[y_var]['y_pred']
    # plt.figure(figsize=(8, 6))
    # plt.scatter(y_actual, y_pred, color='blue', alpha=0.5)
    # plt.plot([y_actual.min(), y_actual.max()], [y_actual.min(), y_actual.max()], 'r--', lw=2)
    # plt.title(f'Предсказанные vs Реальные значения для {y_var} (случайный лес)')
    # plt.xlabel('Реальные значения (Z-нормализованные)')
    # plt.ylabel('Предсказанные значения (Z-нормализованные)')
    # plt.grid(True)
    # plt.tight_layout()
    # plt.show()

In [ ]:
# 8. ANOVA анализ 
print("\nВывод ANOVA для линейной регрессии:")
results_anova = {}
for y_var in dependent_vars:
    data = normalized_df[[y_var] + independent_vars].dropna()
    formula = f'{y_var} ~ USD_RUB + Brent + BTC_USD'
    model = ols(formula, data=data).fit()
    anova_table = anova_lm(model, typ=2)
    results_anova[y_var] = anova_table

    print(f"\nРезультаты ANOVA для {y_var}:")
    print(results_anova[y_var].to_string(index=False))

In [ ]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.utils import resample

# Функция для бутстрэппинга
def bootstrap_regression(X, y, n_bootstraps=1000, random_state=42):
    np.random.seed(random_state)
    coefs_bootstrap = []
    intercepts_bootstrap = []
    
    for _ in range(n_bootstraps):
        # Выборка с возвращением
        X_boot, y_boot = resample(X, y, random_state=random_state)
        model = LinearRegression()
        model.fit(X_boot, y_boot)
        coefs_bootstrap.append(model.coef_)
        intercepts_bootstrap.append(model.intercept_)
    
    # Преобразуем в массив
    coefs_bootstrap = np.array(coefs_bootstrap)
    intercepts_bootstrap = np.array(intercepts_bootstrap)
    
    # Рассчитываем 95%-ные доверительные интервалы
    ci_lower = np.percentile(coefs_bootstrap, 2.5, axis=0)
    ci_upper = np.percentile(coefs_bootstrap, 97.5, axis=0)
    intercept_ci_lower = np.percentile(intercepts_bootstrap, 2.5)
    intercept_ci_upper = np.percentile(intercepts_bootstrap, 97.5)
    
    return coefs_bootstrap.mean(axis=0), ci_lower, ci_upper, intercepts_bootstrap.mean(), intercept_ci_lower, intercept_ci_upper

# Проведение бутстрэппинга для каждой зависимой переменной
dependent_vars = ['TCSG', 'YDEX', 'VKCO', 'RTKM']
independent_vars_full = ['USD_RUB', 'Brent', 'BTC_USD']
independent_vars_tcsg = ['USD_RUB', 'BTC_USD']  # Без Brent для TCSG
results_bootstrap = {}

for y_var in dependent_vars:
    print(f"\nБутстрэппинг для {y_var}:")
    y = normalized_df[y_var].dropna()
    X = normalized_df[independent_vars_full].dropna()
    features = independent_vars_full
    
    common_index = X.index.intersection(y.index)
    X_subset = X.loc[common_index]
    y_subset = y.loc[common_index]
    
    if len(common_index) == 0:
        print(f"Warning: No common indices for {y_var}. Skipping.")
        continue
    
    # Выполняем бутстрэппинг
    mean_coefs, ci_lower, ci_upper, mean_intercept, intercept_ci_lower, intercept_ci_upper = bootstrap_regression(X_subset, y_subset)
    
    # Формируем результаты
    results_bootstrap[y_var] = {
        'features': features,
        'mean_coefs': dict(zip(features, mean_coefs)),
        'ci_lower': dict(zip(features, ci_lower)),
        'ci_upper': dict(zip(features, ci_upper)),
        'mean_intercept': mean_intercept,
        'intercept_ci_lower': intercept_ci_lower,
        'intercept_ci_upper': intercept_ci_upper
    }
    
    # Вывод результатов
    print(f"\nРезультаты для {y_var}:")
    for feature, coef, lower, upper in zip(features, mean_coefs, ci_lower, ci_upper):
        significant = "Значимый" if (lower > 0 or upper < 0) else "Незначимый"
        print(f"{feature}: Коэффициент = {coef:.4f}, 95% CI = [{lower:.4f}, {upper:.4f}] ({significant})")
    print(f"Intercept: Коэффициент = {mean_intercept:.4f}, 95% CI = [{intercept_ci_lower:.4f}, {intercept_ci_upper:.4f}]")

# Формируем таблицу результатов
results_data_bootstrap = []
for y_var in dependent_vars:
    if y_var not in results_bootstrap:
        continue
    row = [y_var]
    for feature in independent_vars_full:
        if feature in results_bootstrap[y_var]['mean_coefs']:
            coef = results_bootstrap[y_var]['mean_coefs'][feature]
            lower = results_bootstrap[y_var]['ci_lower'][feature]
            upper = results_bootstrap[y_var]['ci_upper'][feature]
            significant = "Да" if (lower > 0 or upper < 0) else "Нет"
            row.append(f"{coef:.4f} [{lower:.4f}, {upper:.4f}] ({significant})")
        else:
            row.append("0.0000 [0.0000, 0.0000] (Незначимый)")  # Для Brent в TCSG
    results_data_bootstrap.append(row)

results_df_bootstrap = pd.DataFrame(
    results_data_bootstrap,
    columns=['Компания', 'USD_RUB', 'Brent', 'BTC_USD']
).round(4)

print("\nТаблица результатов бутстрэппинга:")
print(results_df_bootstrap.to_string(index=False))

In [ ]:
# 8. Непараметрический дисперсионный анализ Крускала-Уоллиса
def categorize_factor(series, bins=3):
    labels = ['Низкий', 'Средний', 'Высокий'][:bins]
    return pd.cut(series, bins=bins, labels=labels, include_lowest=True)

normalized_df['USD_RUB_cat'] = categorize_factor(normalized_df['USD_RUB'])
normalized_df['Brent_cat'] = categorize_factor(normalized_df['Brent'])
normalized_df['BTC_USD_cat'] = categorize_factor(normalized_df['BTC_USD'])

from scipy.stats import kruskal

for y_var in dependent_vars:
    print(f"\nТест Крускала-Уоллиса для {y_var}:")
    for factor in ['USD_RUB_cat', 'Brent_cat', 'BTC_USD_cat']:

        groups = [normalized_df[y_var][normalized_df[factor] == level].dropna() for level in normalized_df[factor].unique()]
        stat, p = kruskal(*groups)
        print(f"{factor}: Статистика = {stat:.4f}, p-value = {p:.4f}, "
              f"{'Значимо' if p < 0.05 else 'Незначимо'}")


In [ ]:
# 8. Непараметрический дисперсионный анализ Крускала-Уоллиса
print("\nНепараметрический дисперсионный анализ Крускала-Уоллиса:")

# Создаем список групп для сравнения (например, группы по компаниям)
groups = []
for y_var in dependent_vars:
    groups.append(normalized_df[y_var].dropna())

# Проводим тест Крускала-Уоллиса
stat, p_value = stats.kruskal(*groups)

# Формируем таблицу результатов
results_kruskal = pd.DataFrame({
    'Тест': ['Крускала-Уоллиса'],
    'Статистика': [stat],
    'p-value': [p_value],
    'Вывод': ['Есть значимые различия между группами' if p_value < 0.05 else 'Нет значимых различий между группами']
})

# Выводим результаты
print("\nРезультаты теста Крускала-Уоллиса для сравнения компаний:")
print(results_kruskal.to_string(index=False))

# Дополнительные сравнения для пар групп (пост-хок тест)
if p_value < 0.05:
    print("\nПост-хок анализ (попарные сравнения):")
    from scipy.stats import mannwhitneyu
    from itertools import combinations
    
    # Коррекция Бонферрони для множественных сравнений
    alpha = 0.05
    num_comparisons = len(list(combinations(dependent_vars, 2)))
    corrected_alpha = alpha / num_comparisons
    
    comparisons = []
    for (group1, group2) in combinations(dependent_vars, 2):
        stat, p = mannwhitneyu(normalized_df[group1].dropna(), normalized_df[group2].dropna(), alternative='two-sided')
        significant = "Да" if p < corrected_alpha else "Нет"
        comparisons.append([f"{group1} vs {group2}", stat, p, corrected_alpha, significant])
    
    posthoc_df = pd.DataFrame(comparisons, columns=['Сравнение', 'Статистика', 'p-value', 'Скорректированный alpha', 'Значимо'])
    print(posthoc_df.to_string(index=False))
    
    # Интерпретация
    print("\nИнтерпретация результатов:")
    print("1. Тест Крускала-Уоллиса показывает, есть ли статистически значимые различия между группами в целом.")
    print(f"2. В данном случае p-value = {p_value:.4f}, что {'меньше' if p_value < 0.05 else 'больше'} 0.05.")
    print(f"3. Это означает, что {'существуют' if p_value < 0.05 else 'не существуют'} значимые различия между медианами цен акций компаний.")
    
    if p_value < 0.05:
        print("4. Пост-хок анализ (с поправкой Бонферрони) показывает, между какими именно парами компаний есть различия:")
        for comp in comparisons:
            if comp[4] == "Да":
                print(f"   - {comp[0]}: p-value = {comp[2]:.4f} (различия значимы)")
else:
    print("\nДальнейший пост-хок анализ не требуется, так как не обнаружено значимых различий между группами.")

In [ ]:
# Visualization
#  Updated train_normalized_model to ensure TCSG excludes Brent
def train_normalized_model():
    try:
        independent_vars_tcsg = ['USD_RUB', 'BTC_USD']  # Explicitly exclude Brent for TCSG
        independent_vars_full = ['USD_RUB', 'Brent', 'BTC_USD']
        X_norm = normalized_df[independent_vars_full].dropna()
        X_norm_tcsg = normalized_df[independent_vars_tcsg].dropna()
        results_normalized = {}
        results_normalized_rf = {}
        
        for y_var in dependent_vars:
            y = normalized_df[y_var].dropna()
            if y_var == 'TCSG':
                X_subset_full = X_norm_tcsg
                features_used = independent_vars_tcsg
                # print(f"Training {y_var} with features: {features_used}")
            else:
                X_subset_full = X_norm
                features_used = independent_vars_full
                # print(f"Training {y_var} with features: {features_used}")
            common_index = X_subset_full.index.intersection(y.index)
            if len(common_index) == 0:
                print(f"Warning: No common indices for {y_var}. Skipping.")
                continue
            X_subset = X_subset_full.loc[common_index]
            y_subset = y.loc[common_index]
            
            X_train, X_test, y_train, y_test = train_test_split(X_subset, y_subset, test_size=0.2, random_state=42)
            # print(f"{y_var} X_train columns: {list(X_train.columns)}")
            # print(f"{y_var} X_test columns: {list(X_test.columns)}")
            
            # Linear Regression
            model_lr = LinearRegression()
            model_lr.fit(X_train, y_train)
            y_pred_lr = model_lr.predict(X_test)
            mae_lr = np.mean(np.abs(y_test - y_pred_lr))
            mse_lr = np.mean((y_test - y_pred_lr) ** 2)
            
            if y_var == 'TCSG':
                coefs = dict(zip(independent_vars_tcsg, model_lr.coef_))
                coefs['Brent'] = 0.0  # Add Brent with 0 coefficient for consistency
            else:
                coefs = dict(zip(independent_vars_full, model_lr.coef_))
            
            results_normalized[y_var] = {
                'y_actual': y_test,
                'y_pred': y_pred_lr,
                'X_train': X_train,
                'X_test': X_test,
                'model': model_lr,
                'coefficients': coefs,
                'intercept': model_lr.intercept_,
                'MAE': mae_lr,
                'MSE': mse_lr
            }
            
            # Random Forest
            model_rf = RandomForestRegressor(n_estimators=50, random_state=42)
            model_rf.fit(X_train, y_train)
            y_pred_rf = model_rf.predict(X_test)
            mae_rf = np.mean(np.abs(y_test - y_pred_rf))
            mse_rf = np.mean((y_test - y_pred_rf) ** 2)
            
            if y_var == 'TCSG':
                importances = dict(zip(independent_vars_tcsg, model_rf.feature_importances_))
                importances['Brent'] = 0.0  # Add Brent with 0 importance for consistency
            else:
                importances = dict(zip(independent_vars_full, model_rf.feature_importances_))
            
            results_normalized_rf[y_var] = {
                'y_actual': y_test,
                'y_pred': y_pred_rf,
                'X_train': X_train,
                'X_test': X_test,
                'model': model_rf,
                'feature_importances': importances,
                'MAE': mae_rf,
                'MSE': mse_rf
            }
        
        return results_normalized, results_normalized_rf
    except Exception as e:
        print(f"Error in train_normalized_model: {e}")
        return {}, {}

def predict_prices(usd_rub, brent, btc_usd, data_type, model_type):
    try:
        predictions = {}
        results = results_normalized if model_type == 'Линейная регрессия' else results_normalized_rf
        ylabel = 'Цена акций (Z-нормализованная)' if data_type == 'Z-нормализованные' else 'Цена акций (рубли)'
        
        # Convert inputs to Z-normalized values
        usd_rub_norm = usd_rub if data_type == 'Z-нормализованные' else (usd_rub - cleaned_df['USD_RUB'].mean()) / cleaned_df['USD_RUB'].std()
        brent_norm = brent if data_type == 'Z-нормализованные' else (brent - cleaned_df['Brent'].mean()) / cleaned_df['Brent'].std()
        btc_usd_norm = btc_usd if data_type == 'Z-нормализованные' else (btc_usd - cleaned_df['BTC_USD'].mean()) / cleaned_df['BTC_USD'].std()
        
        for y_var in dependent_vars:
            if y_var not in results:
                print(f"Warning: No model for {y_var}. Skipping prediction.")
                continue
                
            model = results[y_var]['model']
            expected_features = list(results[y_var]['X_test'].columns)
            
            # Prepare input data based on expected features
            if y_var == 'TCSG':
                input_data = pd.DataFrame({
                    'USD_RUB': [usd_rub_norm],
                    'BTC_USD': [btc_usd_norm]
                })
            else:
                input_data = pd.DataFrame({
                    'USD_RUB': [usd_rub_norm],
                    'Brent': [brent_norm],
                    'BTC_USD': [btc_usd_norm]
                })
            
            # Ensure columns match exactly what model expects
            input_data = input_data[expected_features]
            
            prediction = model.predict(input_data)[0]
            
            # Denormalize prediction if output is in rubles
            if data_type == 'Реальные (рубли)':
                prediction = prediction * cleaned_df[y_var].std() + cleaned_df[y_var].mean()
                prediction = max(0, prediction)  # Ensure non-negative prices
            predictions[y_var] = prediction
        return predictions, ylabel
    except Exception as e:
        print(f"Error in predict_prices: {e}")
        return {}, 'Цена акций'

def update_plot(usd_rub, brent, btc_usd, data_type, model_type):
    try:
        predictions, ylabel = predict_prices(usd_rub, brent, btc_usd, data_type, model_type)
        if not predictions:
            print("No predictions available.")
            return
        
        # Create bar chart
        plt.figure(figsize=(8, 6))
        companies = list(predictions.keys())
        values = [predictions[company] for company in companies]
        bars = plt.bar(companies, values, color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'])
        plt.title(f'Прогнозируемые цены акций ({data_type}, {model_type})', fontsize=14)
        plt.xlabel('Компания')
        plt.ylabel(ylabel)
        plt.grid(True, axis='y')
        for bar, value in zip(bars, values):
            plt.text(bar.get_x() + bar.get_width()/2, value + 0.02 * value, f'{value:.2f}', ha='center', va='bottom')
        plt.tight_layout()
        plt.show()
        
        # print(f"\nПрогнозы ({data_type}, {model_type}):")
        # for k, v in predictions.items():
        #     print(f"{k}: {round(v, 2)}")

        # Debug output
        # results = results_normalized if model_type == 'Линейная регрессия' else results_normalized_rf
        # print(f"\nРезультаты модели ({data_type}, {model_type}):")
        # for y_var in dependent_vars:
        #     if y_var in results:
        #         print(f"\n{y_var}:")
        #         if model_type == 'Линейная регрессия':
        #             print(f"  USD_RUB: {results[y_var]['coefficients']['USD_RUB']:.4f}")
        #             print(f"  Brent: {results[y_var]['coefficients']['Brent']:.4f}")
        #             print(f"  BTC_USD: {results[y_var]['coefficients']['BTC_USD']:.4f}")
        #             print(f"  Intercept: {results[y_var]['intercept']:.4f}")
        #         else:
        #             print(f"  USD_RUB importance: {results[y_var]['feature_importances']['USD_RUB']:.4f}")
        #             print(f"  Brent importance: {results[y_var]['feature_importances']['Brent']:.4f}")
        #             print(f"  BTC_USD importance: {results[y_var]['feature_importances']['BTC_USD']:.4f}")
        #         print(f"  MAE: {results[y_var]['MAE']:.4f}")
        #         print(f"  MSE: {results[y_var]['MSE']:.4f}")
        #     else:
        #         print(f"{y_var}: No model data available.")
    
    except Exception as e:
        print(f"Ошибка в интерактивной визуализации: {e}")

# Sliders for real data
usd_rub_slider_ruble = FloatSlider(
    min=cleaned_df['USD_RUB'].min(), max=cleaned_df['USD_RUB'].max(), step=0.1,
    value=cleaned_df['USD_RUB'].mean(), description='USD/RUB (руб):'
)
brent_slider_ruble = FloatSlider(
    min=cleaned_df['Brent'].min(), max=cleaned_df['Brent'].max(), step=0.1,
    value=cleaned_df['Brent'].mean(), description='Brent (долл.):'
)
btc_usd_slider_ruble = FloatSlider(
    min=cleaned_df['BTC_USD'].min(), max=150000, step=100,
    value=cleaned_df['BTC_USD'].mean(), description='BTC/USD (долл.):'
)

# Sliders for Z-normalized data
btc_mean = cleaned_df['BTC_USD'].mean()
btc_std = cleaned_df['BTC_USD'].std()
btc_max_norm = (150000 - btc_mean) / btc_std
btc_min_norm = (cleaned_df['BTC_USD'].min() - btc_mean) / btc_std

usd_rub_slider_norm = FloatSlider(
    min=normalized_df['USD_RUB'].min(), max=normalized_df['USD_RUB'].max(), step=0.1,
    value=normalized_df['USD_RUB'].mean(), description='USD/RUB (Z):'
)
brent_slider_norm = FloatSlider(
    min=normalized_df['Brent'].min(), max=normalized_df['Brent'].max(), step=0.1,
    value=normalized_df['Brent'].mean(), description='Brent (Z):'
)
btc_usd_slider_norm = FloatSlider(
    min=btc_min_norm, max=btc_max_norm, step=0.1,
    value=normalized_df['BTC_USD'].mean(), description='BTC/USD (Z):'
)

# Dropdowns
data_type_dropdown = Dropdown(
    options=['Z-нормализованные', 'Реальные (рубли)'],
    value='Реальные (рубли)',
    description='Тип данных:'
)
model_type_dropdown = Dropdown(
    options=['Линейная регрессия', 'Случайный лес'],
    value='Линейная регрессия',
    description='Модель:'
)

def update_sliders(data_type, model_type):
    try:
        if data_type == 'Z-нормализованные':
            interact(
                update_plot,
                usd_rub=usd_rub_slider_norm,
                brent=brent_slider_norm,
                btc_usd=btc_usd_slider_norm,
                data_type=fixed(data_type),
                model_type=fixed(model_type)
            )
        else:
            interact(
                update_plot,
                usd_rub=usd_rub_slider_ruble,
                brent=brent_slider_ruble,
                btc_usd=btc_usd_slider_ruble,
                data_type=fixed(data_type),
                model_type=fixed(model_type)
            )
    except Exception as e:
        print(f"Error in update_sliders: {e}")

# First train the models
results_normalized, results_normalized_rf = train_normalized_model()

# Then run the interactive widget
interact(update_sliders, data_type=data_type_dropdown, model_type=model_type_dropdown)